In [1]:
using LinearAlgebra, Plots, SetRounding, ColorBitstring, Test

import Base: getindex, setindex!, size, *, \

### Reflection

In [ ]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

function size(Q::Reflection)
    (length(Q.v),length(Q.v))
end

# getindex(Q, k, j) is synonym for Q[k,j]
function getindex(Q::Reflection, k::Int, j::Int)
    # Implement Q[k,j] == (I - 2v*v')[k,j] but using O(1) operations.
    # Hint: the function `conj` gives the complex-conjugate
    if k == j
        1 - 2Q.v[k]*conj(Q.v[j])
    else
        - 2Q.v[k]*conj(Q.v[j])
    end
end
function *(Q::Reflection, x::AbstractVector)
    # Implement Q*x, equivalent to (I - 2v*v')*x
    # but using only O(n) operations
    x - 2*Q.v * dot(Q.v,x) # (Q.v'*x) also works instead of dot
end

### Reflections

In [ ]:
struct Reflections{T} <: AbstractMatrix{T}
    V::Matrix{T}
end

size(Q::Reflections) = (size(Q.V,1), size(Q.V,1))

function *(Q::Reflections, x::AbstractVector)
    # Apply Q in O(mn) operations by applying
    # the reflection corresponding to each column of Q.V to x
    m,n = size(Q.V)
    for j = n:-1:1
        x = Reflection(Q.V[:, j]) * x
    end

    x
end

function getindex(Q::Reflections, k::Int, j::Int)
    # Return Q[k,j] in O(mn) operations (hint: use *)
    T = eltype(Q.V)
    m,n = size(Q)
    ej = zeros(T, m)
    ej[j] = one(T)
    return (Q*ej)[k]
end

### Householder reflection

In [ ]:
function householderreflection(s::Bool, x::AbstractVector)
    # Return a `Reflection` corresponding to a Householder reflection
    y = copy(x) # don't modify `x`
    if s
        y[1] -= norm(x)
    else
        y[1] += norm(x)
    end
    Reflection(y)
end

### QR (Householder)
$O(mn^{2})$ using `Reflection` and `Reflections`

In [ ]:
function householderqr(A)
    T = eltype(A)
    m,n = size(A)
    if n > m
        error("More columns than rows is not supported")
    end

    R = zeros(T, m, n)
    Q = Reflections(zeros(T, m, n))
    Aⱼ = copy(A)

    for j = 1:n
        # Rewrite householder QR to use Reflection and Reflections,
        # in a way that one achieves O(mn^2) operations
        𝐚₁ = Aⱼ[:,1] # first columns of Aⱼ
        Q₁ = householderreflection(𝐚₁[1] < 0, 𝐚₁)
        Q₁Aⱼ = Q₁*Aⱼ
        α,𝐰 = Q₁Aⱼ[1,1],Q₁Aⱼ[1,2:end]
        Aⱼ₊₁ = Q₁Aⱼ[2:end,2:end]

        # populate returned data
        R[j,j] = α
        R[j,j+1:end] = 𝐰

        Q.V[j:end, j] = Q₁.v

        Aⱼ = Aⱼ₊₁ # this is the "induction"
    end
    Q,R
end

In [39]:
function myqr(A)
    T = eltype(A)
    m, n = size(A)
    m >= n || error("m < n")

    R = copy(A)
    Q = Matrix(one(T)*I, m, m)
    for j = 1:n 
        y = R[j:end, j]
        y[1] -= norm(y)
        w = y / norm(y)

        Qₓ = I - 2 * w * w'
        R[j:end, j:end] = Qₓ * R[j:end, j:end]
        Q[:, j:end] = Q[:, j:end] * Qₓ
    end

    Q, R
end

myqr (generic function with 1 method)

### Cholesky
for tridiagonal matrices, use `SymTridiagonal`

`SymTridiagonal(dv, eu) == Tridiagonal(ev, dv, ev)`


In [ ]:
# Return a Bidiagonal L such that L'L == A (up to machine precision)
function mycholesky(A::SymTridiagonal)
    d = A.dv # diagonal entries of A
    u = A.ev # sub/super-diagonal entries of A
    T = float(eltype(A)) # return type, make float in case A has Ints
    n = length(d)
    ld = zeros(T, n) # diagonal entries of L
    ll = zeros(T, n-1) # sub-diagonal entries of L

    # Populate the diagonal entries ld and
    # the sub-diagonal entries ll of L so that L*L' ≈ A
    ld[1] = sqrt(d[1])
    for k = 1:n-1
        ll[k] = u[k]/ld[k]
        ld[k+1] = sqrt(d[k+1]-ll[k]^2)
    end

    Bidiagonal(ld, ll, :L)
end

In [4]:
function mycholesky(A)
    T = eltype(A)
    n, m = size(A)
    n == m || error("Not square")
    A == A' || error("Not symmetric")

    L = LowerTriangular(zeros(T, n, n))
    Aⱼ = copy(A)
    for j = 1:n 
        α, 𝐯 = Aⱼ[1, 1], Aⱼ[2:end, 1]
        α > 0 || error("Not SPD")

        L[j, j] = sqrt(α)
        L[j+1:end, j] = 𝐯 / sqrt(α)

        Aⱼ = Aⱼ[2:end, 2:end] - 𝐯*𝐯'/α
    end
    L
end

mycholesky (generic function with 1 method)

### SVD best rank approximation

In [ ]:
function svdcompress(A::Matrix, k::Integer)
    U,σ,V = svd(A)
    U[:,1:k] * Diagonal(σ[1:k]) * V[:,1:k]'
end

### Numerical rank
the smallest integer `k` s.t. `opnorm(A - svdcompress(A, k)) ≤ ε`

In [ ]:
function svdcompress_rank(A::Matrix, ε::Real)
    # Determine and return rank-k approximation
    σ = svdvals(A)
    for k = 1:length(σ)
        if σ[k] ≤ ε
            return k-1
        end
    end
    return length(σ)
end

### SVD pseudo-inverse $A^+$

In [ ]:
function pseudoinv(A)
    m,n = size(A)
    m == n || error("A must be square")
    tol = 1E-15 # threshold below which we assume a singular value is zero
    # Construct and return the pseudo inverse of A
    U,σ,V = svd(A)
    r = 0
    for k = 1:length(σ)
        if σ[k] > tol
            r += 1
        end
    end
    V[:,1:r] * Diagonal(inv.(σ[1:r])) * U[:,1:r]'
end

Vandermonde matrix

In [ ]:
function vandermonde(𝐱, n) # 𝐱 = [x_1,…,x_m]
    # [𝐱[j]^k for j = 1:m, k = 0:n-1]
    𝐱 .^ (0:(n-1))'
end

`ql(A)`, where `Q` is orthogonal and `L` is lower triangular

In [ ]:
function lowerhouseholderreflection(x)
    ## Implement the householder reflector
    y = copy(x)
    y[end] += norm(x)
    w = y/norm(y)
    I - 2w*w'
end

function ql(A) # assume A is square
    m,n = size(A)
    m == n || error("not square")
    # Create Q and L such that Q'Q == I and L is lower triangular
    L = copy(A)
    Q = Matrix(1.0I, n, n)
    for j = n:-1:2
        Qⱼ = lowerhouseholderreflection(L[1:j, j])
        L[1:j, 1:j] = Qⱼ * L[1:j, 1:j]
        Q[:,1:j] = Q[:,1:j]*Qⱼ
    end
    Q,L
end

`lq(A)`, where `L` is lower triangular and `Q` is orthogonal

consider a Householder reflection s.t. $𝐱^⊤ Q = \|𝐱\|𝐞_1^⊤$

In [2]:
function lq(A)
    m,n = size(A)
    m == n || error("not square")
    # Create Q and L such that A = L*Q, Q'Q == I and L is lower triangular
    L = copy(A)
    Q = Matrix(1.0I, n, n)
    for k = 1:n-1
        y = L[k, k:end]
        y[1] -= norm(y)
        w = y / norm(y)
        Qₖ = I - 2 * w * w'
        L[k:end, k:end] = L[k:end, k:end] * Qₖ
        Q[k:end, :] = Qₖ * Q[k:end, :]
    end
    L,Q
end

A = [1.0 2 3; 1 4 9; 1 1 1]
L,Q = lq(A)
@test Q'Q ≈ I
@test L*Q ≈ A
@test L ≈ tril(L) # it is acceptable to have small non-zero entries in L

Test Passed